# Import libraries

In [1]:
import pickle
import joblib
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold

from matplotlib import pyplot
from collections import Counter
from xgboost import XGBClassifier
from plotly.subplots import make_subplots
from imblearn.over_sampling import SMOTENC
from mlxtend.plotting import plot_confusion_matrix

# Load and check data

In [2]:
df_data = pd.read_csv('dataset/HR_comma_sep.csv')
df_data.head()

satisfaction_level  last_evaluation  number_project  average_montly_hours  \
0                0.38             0.53               2                   157   
1                0.80             0.86               5                   262   
2                0.11             0.88               7                   272   
3                0.72             0.87               5                   223   
4                0.37             0.52               2                   159   

   time_spend_company  Work_accident  left  promotion_last_5years Department  \
0                   3              0     1                      0      sales   
1                   6              0     1                      0      sales   
2                   4              0     1                      0      sales   
3                   5              0     1                      0      sales   
4                   3              0     1                      0      sales   

   salary  
0     low  
1  medium  
2  medium  
3     low  
4     low

In [3]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   satisfaction_level     14999 non-null  float64
 1   last_evaluation        14999 non-null  float64
 2   number_project         14999 non-null  int64  
 3   average_montly_hours   14999 non-null  int64  
 4   time_spend_company     14999 non-null  int64  
 5   Work_accident          14999 non-null  int64  
 6   left                   14999 non-null  int64  
 7   promotion_last_5years  14999 non-null  int64  
 8   Department             14999 non-null  object 
 9   salary                 14999 non-null  object 
dtypes: float64(2), int64(6), object(2)
memory usage: 1.1+ MB


# Check whether outlier exist

In [4]:
# draw histogram of some features
# df_data.hist()
# plt.tight_layout()
# pyplot.show()

In [8]:
# draw histogram of some features
fig = make_subplots(rows=3, cols=3)
trace0 = go.Histogram(x=df_data['satisfaction_level'], histfunc="count", name='Satisfaction Level')
trace1 = go.Histogram(x=df_data['last_evaluation'], histfunc="count", name='Last Evalution')
trace2 = go.Histogram(x=df_data['number_project'], histfunc="count", name='Number of Project')
trace3 = go.Histogram(x=df_data['average_montly_hours'], histfunc="count", name='Average Monthly Hours')
trace4 = go.Histogram(x=df_data['time_spend_company'], histfunc="count", name='Time Spend Company')
trace5 = go.Histogram(x=df_data['Work_accident'], histfunc="count", name='Work Accident')
trace6 = go.Histogram(x=df_data['promotion_last_5years'], histfunc="count", name='Promotion Last 5 Years')
# trace7 = go.Histogram(x=df_data['Department'], histfunc="count", name='Department')
# trace8 = go.Histogram(x=df_data['salary'], histfunc="count", name='Salary')

fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)
fig.append_trace(trace2, 1, 3)
fig.append_trace(trace3, 2, 1)
fig.append_trace(trace4, 2, 2)
fig.append_trace(trace5, 2, 3)
fig.append_trace(trace6, 3, 1)
# fig.append_trace(trace7, 3, 2)
# fig.append_trace(trace8, 3, 3)
fig.update_layout(autosize=False, width=1500, height=800, margin=dict(l=10, r=10, b=10, t=10))
fig.show()

In [14]:
df_data.value_counts('salary').to_frame().style.background_gradient(cmap='plasma_r')

In [15]:
df_data.value_counts('Department').to_frame().style.background_gradient(cmap='plasma_r')

# Split data into input (features) and output (target)

In [16]:
y = df_data['left']
X = df_data.drop('left', axis=1)

# Check whether imbalance exist

In [24]:
# compare portion of each class 
class_name = []
class_perc = []
counter = Counter(y.values)
for k,v in counter.items():
    per = v / len(y.values) * 100
    class_name.append(k)
    class_perc.append(round(per, 2))
    print( ' Class=%d, Count=%d, Percentage=%.3f%% ' % (k, v, per))

 Class=1, Count=3571, Percentage=23.808% 
 Class=0, Count=11428, Percentage=76.192% 


In [29]:
fig = px.bar(
    x=['Left', 'Stay'],
    y=class_perc,
    color=class_perc,
    text=class_perc,
    width=500,
    labels={'x':'Class', 'y':'Percentage'}
)

fig.show()

In [8]:
# # remove imbalance data problem by oversample with SMOTENC algorithm (SMOTE only use for numerical features)
# oversample = SMOTENC(categorical_features=[7, 8], random_state=42)
# X, y = oversample.fit_resample(X, y)
# counter = Counter(y.values)
# for k,v in counter.items():
#     per = v / len(y.values) * 100
#     print( ' Class=%d, Count=%d, Percentage=%.3f%% ' % (k, v, per))

# Train some ML models with data transformation in pipeline

In [9]:
# get categorical and numerical columns
numerical_ix = X.select_dtypes(include=['int64', 'float64']).columns
categorical_ix = X.select_dtypes(include=['object', 'bool']).columns

In [10]:
# define ColumnTransformer to perform transform data
trans_list = [('cat', OneHotEncoder(), categorical_ix), ('num', StandardScaler(), numerical_ix)]
col_trans = ColumnTransformer(transformers=trans_list)

In [12]:
# create a dictionary of models
model_dict = {
                'LogisticRegression': LogisticRegression(max_iter=500, class_weight='balanced'), 
                'GaussianNB': GaussianNB(), 
                'KNeighborsClassifier': KNeighborsClassifier(), 
                'DecisionTreeClassifier': DecisionTreeClassifier(class_weight='balanced'),
                'SVM': svm.SVC(class_weight='balanced'),
                'RandomForestClassifier': RandomForestClassifier(class_weight='balanced'),
            }

In [13]:
# evaluate models in dictionary using cross validation with RepeatedStratifiedKFold
model_scores = []
model_names = []
for name, model in model_dict.items():
    pipeline = Pipeline(steps=[('preparation', col_trans), ('model', model)])
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=42)
    scores = cross_val_score(pipeline, X, np.ravel(y), scoring='accuracy', cv=cv, n_jobs=-1)
    model_scores.append(scores)
    model_names.append(name)
    print('*'*10 + name + '*'*10)
    print(np.mean(scores))
    print(np.std(scores))

**********LogisticRegression**********
0.7598638247720703
0.00971910352647794
**********GaussianNB**********
0.6595039092728485
0.014364290029621824
**********KNeighborsClassifier**********
0.9455297175895042
0.005250667653662472
**********DecisionTreeClassifier**********
0.9819988258839224
0.003200161402127842
**********SVM**********
0.9559569134978875
0.004905774407557175
**********RandomForestClassifier**********
0.9913393907049142
0.0022560674653345505


In [14]:
# display evaluation result on violin chart
fig = go.Figure()
for model, score in zip(model_names, model_scores):
    fig.add_trace(go.Violin(
                            y=score,
                            name=model,
                            box_visible=True,
                            meanline_visible=True)
                        )
fig.update_layout(
    autosize=False,
    width=1000,
    height=800,
    margin=dict(l=10, r=10, b=10, t=10),
    # paper_bgcolor="LightSteelBlue",
)
fig.show()

## Try to train model with XGBoost algorithm

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# encoder = OrdinalEncoder()
# encoder.fit(X_train[categorical_ix])
# X_train[categorical_ix] = encoder.transform(X_train[categorical_ix])
# X_test[categorical_ix] = encoder.transform(X_test[categorical_ix])

# lb_encoder = LabelEncoder()
# lb_encoder.fit(y_train)
# y_train = lb_encoder.transform(y_train)
# y_test = lb_encoder.transform(y_test)

model = XGBClassifier(use_label_encoder=False, verbosity=0, class_weight='balanced')
pipeline = Pipeline(steps=[('trans', col_trans), ('model', model)])

pipeline.fit(X_train, np.ravel(y_train))
predictions = pipeline.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print('Accuracy: %.2f%%' % (accuracy*100))

Accuracy: 98.62%


## Gridseach RandomForestClassifier model

In [16]:
rf_model = RandomForestClassifier(random_state=42, class_weight='balanced')
pipeline = Pipeline(steps=[('preparation', col_trans), ('rf', rf_model)])
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=42)

In [17]:
# param_grid = { 
#     'rf__n_estimators': np.arange(5,20),
#     'rf__max_features': ['auto', 'sqrt', 'log2'],
#     'rf__max_depth' : np.arange(20,40),
#     'rf__criterion' :['gini', 'entropy'],
#     # 'rf__min_samples_split': np.arange(0,10)
# }

param_grid = {'rf__n_estimators': np.arange(50, 150, 10),
               'rf__max_features': ['auto', 'sqrt', 'log2'],
               'rf__max_depth': np.arange(10, 100, 10),
               'rf__min_samples_split': [2, 5, 10],
               'rf__min_samples_leaf': [1, 2, 4],
               'rf__criterion' :['gini', 'entropy'],
               'rf__bootstrap': [True, False]}


In [18]:
grid_pipeline = GridSearchCV(pipeline, param_grid, scoring= 'accuracy', n_jobs=-1, cv=cv)

In [22]:
results = grid_pipeline.fit(X, np.ravel(y))

In [23]:
print( ' Best Mean Accuracy: %.3f ' % results.best_score_)
print( ' Best Config: %s ' % results.best_params_)

 Best Mean Accuracy: 0.992 
 Best Config: {'rf__bootstrap': False, 'rf__criterion': 'entropy', 'rf__max_depth': 30, 'rf__max_features': 'auto', 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 2, 'rf__n_estimators': 70} 


## Use the best model to make prediction on test set

In [24]:
# actually, test set is used to train model. So, this is not really valuable.
y_pred = grid_pipeline.predict(X_test)

# create confusion matrix
cm = confusion_matrix(y_test, y_pred)
# display classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3428
           1       1.00      1.00      1.00      1072

    accuracy                           1.00      4500
   macro avg       1.00      1.00      1.00      4500
weighted avg       1.00      1.00      1.00      4500



In [25]:
# display confusion matrix
fig, ax = plot_confusion_matrix(conf_mat=cm,
                                show_absolute=True,
                                show_normed=True,
                                colorbar=True)
plt.show()

## Use the best model to make prediction on new single sample

In [26]:
# create new sample
new_sample = [[0.41, 0.43, 3, 153, 3, 1, 1, 'sales', 'medium']]
df_new_sample = pd.DataFrame(new_sample)
df_new_sample.columns = ['satisfaction_level', 'last_evaluation', 'number_project', 'average_montly_hours', 'time_spend_company', 'Work_accident', 'promotion_last_5years', 'Department', 'salary']

In [27]:
# get only predicted class

# way 1
grid_pipeline.predict(df_new_sample)

# way 2, same result as way 1
# grid_pipeline.best_estimator_.predict(df_new_sample)

array([0], dtype=int64)

In [28]:
# get predicted class and it's probably coresponding
pd.DataFrame(grid_pipeline.predict_proba(df_new_sample)*100, columns=grid_pipeline.classes_)

0         1
0  98.571429  1.428571

In [29]:
# compare grid_pipeline and grid_pipeline.best_estimator_
# print(grid_pipeline.best_estimator_)
# print(grid_pipeline)

# Save the best model

In [30]:
# way 1
pickle.dump(grid_pipeline, open('model.pkl', 'wb'))
# pickle.dump(grid_pipeline.best_estimator_, open('model.pkl', 'wb'))

# way 2, same result as way 1
joblib.dump(grid_pipeline, 'model.pkl')
# joblib.dump(grid_pipeline.best_estimator_, 'model.pkl')

['model.pkl']

# Load saved model

In [31]:
# way 1
model = pickle.load(open('model.pkl', 'rb'))
# way 2, same result as way 1
model = joblib.load('model.pkl')

In [33]:
# use loaded model to make prediction
class_id = model.predict(df_new_sample)[0]
print(class_id)

0
